# Homework 6

## Problem 1: Encrypted R1CS

Given an R1CS of the form:

$$
L\mathbf{[\vec{s}]_1}\odot R\mathbf{[\vec{s}]_2} = O\mathbf{[\vec{s}]_1}\odot[\vec{G_2}]_2
$$

Where $L$, $R$ and $O$ are $n \times m$ matrices of field elements and $s$ is a vector of $G_1$ or $G_2$ points.

Write Python code that verifies the formula.

You can check the equality of $G_{12}$ points in Python this way:

```python
a = pairing(multiply(G2, 5), multiply(G1, 8))
b = pairing(multiply(G2, 10), multiply(G1, 4))
assert eq(a, b)
```

**Hint:** Each row of the matrices is a separate pairing.  
**Hint:** When you get $s$ encrypted with both $G_1$ and $G_2$ generators, you don't know whether or not they have the same discrete logarithm. However, it is straightforward to check using another equation.

Solidity cannot multiply G2 points, do this assignment in Python.

### Answer

Let both $L$, $R$, $O$ be matrices like:

$$
\begin{bmatrix}
a_{1,1} & a_{1,2} & \cdots & a_{1,m} \\
a_{2,1} & a_{2,2} & \cdots & a_{2,m} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n,1} & a_{n,2} & \cdots & a_{n,m} \\
\end{bmatrix}
$$

And $s$ be a vector like:
$$
\begin{bmatrix}
s_1 & s_2 & \cdots & s_m
\end{bmatrix}
$$

We can effectively multiply them if we transpose $s$:
$$
\begin{equation}
\begin{bmatrix}
a_{1,1} & a_{1,2} & \cdots & a_{1,m} \\
a_{2,1} & a_{2,2} & \cdots & a_{2,m} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n,1} & a_{n,2} & \cdots & a_{n,m} \\
\end{bmatrix}
\times
\begin{bmatrix}
s_1 \\ s_2 \\ \vdots \\ s_m
\end{bmatrix}
\end{equation}
$$

Notice how we can refactor the expression above as a sequence of Hadamard product and addition:

$$
\begin{align*}
\begin{bmatrix}
a_{1,1} & a_{1,2} & \cdots & a_{1,m} \\
a_{2,1} & a_{2,2} & \cdots & a_{2,m} \\
\vdots  & \vdots  & \ddots & \vdots \\
a_{n,1} & a_{n,2} & \cdots & a_{n,m} \\
\end{bmatrix}
\times
\begin{bmatrix}
s_1 \\ s_2 \\ \vdots \\ s_m
\end{bmatrix} &=

\begin{bmatrix}
a_{1,1} \\
a_{2,1} \\
\vdots  \\
a_{n,1} \\
\end{bmatrix}
\odot
\begin{bmatrix}
s_1    \\
s_1    \\
\vdots \\
s_1    \\
\end{bmatrix}

+

\begin{bmatrix}
a_{1,2} \\
a_{2,2} \\
\vdots  \\
a_{n,2} \\
\end{bmatrix}
\odot
\begin{bmatrix}
s_2    \\
s_2    \\
\vdots \\
s_2    \\
\end{bmatrix}

+
\cdots
+

\begin{bmatrix}
a_{1,m} \\
a_{2,m} \\
\vdots  \\
a_{n,m} \\
\end{bmatrix}
\odot
\begin{bmatrix}
s_m    \\
s_m    \\
\vdots \\
s_m    \\
\end{bmatrix} \\
\\
&= 

\begin{bmatrix}
\sum\limits_{i=1}^{m}s_ia_{1,i} \\
\sum\limits_{i=1}^{m}s_ia_{2,i} \\
\vdots \\
\sum\limits_{i=1}^{m}s_ia_{n,i} \\
\end{bmatrix}

\end{align*}
$$

In [28]:
import random
import numpy as np
from py_ecc.bn128 import G1, Z1, G2, Z2, curve_order, multiply, add, pairing, eq

"""
It is not possible to call `multiply` with negative numbers.
To circumvent this limitation, this helper function computes the modular inverse
if the input relative to the curve order to achieve the same result.
"""
def normalized_multiply(G, x):
    return multiply(G, x if x >= 0 else curve_order + x)

def compute_vector(A, s, acc0):
    As = np.array([])
    for i, _ in enumerate(s):
        acc = acc0
        for j in range(len(A)):
            acc = add(acc, normalized_multiply(s[i], A[j][i]))
        np.append(As, acc)
    return As

def verify(L, R, O, s1, s2):
    Ls1 = compute_vector(L, s1, Z1)
    Rs2 = compute_vector(R, s2, Z2)
    Os1 = compute_vector(O, s1, Z1)

    for _, i in Ls1:
        assert eq(pairing(Rs2[i], Ls1[i]), pairing(Os1[i], G2))


# This is from week 5 homework, problem 1:

L = np.array([
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 5, 0, 0, 0],
])

R = np.array([
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
])

O = np.array([
    [0, 0, 0,  0,  1, 0, 0,   0],
    [0, 0, 0,  0,  0, 1, 0,   0],
    [0, 0, 0,  0,  0, 0, 1,   0],
    [0, 0, 0,  0,  0, 0, 0,   1],
    [0, 1, 0, 10, -1, 0, 4, -13],
])

x = random.randint(1,1000)
y = random.randint(1,1000)

out = 5*x**3 -4*y**2*x**2 + 13*x*y**2 + x**2 - 10*y

v1 = x*x
v2 = y*y
v3 = v1*v2
v4 = v2*x

w = np.array([1, out, x, y, v1, v2, v3, v4])

toGn = lambda G: lambda x: normalized_multiply(G, x)
toG1 = toGn(G1)
toG2 = toGn(G2)

s1 = np.array([toG1(wi) for wi in w])
s2 = np.array([toG2(wi) for wi in w])

verify(L, R, O, s1, s2)